## Load and validate final model


In [1]:
pwd

'/home/ucsddse230/work/final_project/post_azure'

In [2]:
import pyspark.ml.feature 
from pyspark.ml.feature import CountVectorizer
import pyspark.sql.functions  as psf
from pyspark.sql import Window 
from pyspark.sql.types  import DateType
import pandas as pd
#pd.set_option('display.max_colwidth', -1)
from pyspark.sql.types import Row, StructField, StructType, StringType, IntegerType , DateType, ArrayType

In [3]:
#sc.stop()
import findspark
findspark.init()
from pyspark.sql import SQLContext
from pyspark.sql.types import Row, StructField, StructType, StringType, IntegerType , DateType, ArrayType
from pyspark.sql import SparkSession
from pyspark import SparkContext ,SparkConf
SparkContext.setSystemProperty('spark.executor.memory', '6g')
#SparkContext.setSystemProperty('spark.rdd.compress', True)
SparkContext.setSystemProperty('spark.driver.memory', '16g')
sc = SparkContext(master="local[4]")
print(sc)


<SparkContext master=local[4] appName=pyspark-shell>


In [4]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression, LinearSVC,NaiveBayes
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.feature import HashingTF, Tokenizer
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import RandomForestClassifier

In [5]:
save_file = 'MyMLP.pth'
train_path='/home/ucsddse230/work/final_project/post_azure/training_features.parquet'
test_path='/home/ucsddse230/work/final_project/post_azure/testing_features.parquet'
dl_path='/home/ucsddse230/work/final_project/dl_model/'

In [6]:
spark = SparkSession.builder.getOrCreate()

In [7]:
train_data= spark.read.parquet(train_path)
test_data= spark.read.parquet(test_path)

## happy modeling
>please explore the the multilayer perceptron here and many others
https://spark.apache.org/docs/2.3.0/ml-classification-regression.html


In [8]:
test_data.columns

['HADM_ID', 'SUBJECT_ID', 'label', 'tokens_clean', 'features']

In [9]:
print(train_data.count(),len(train_data.columns))
print(test_data.count(),len(test_data.columns))

40795 5
10318 5


In [10]:
pos_count = train_data[train_data.label==1].count()
neg_count = train_data[train_data.label==0].count()

In [11]:
from utils import train, evaluate, display_metrics, classification_metrics

In [12]:
#Uncomment below line to install torch
#!pip install torch

In [13]:
#DL libraries and pandas and numpy
import numpy as np
import pandas as pd
from scipy import sparse
import torch
from torch.utils.data import TensorDataset, Dataset


In [14]:
#Convert to Pandas dataframe
test_df = test_data.toPandas()

In [15]:
print(test_df.shape)

(10318, 5)


In [16]:
test_y = np.array(test_df['label'].values)

In [17]:
#Create tensor datasets for MLP


#test dataset
target = torch.from_numpy(test_y.astype('long')).view(-1)

series = test_df['features'].apply(lambda x : np.array(x.toArray())).as_matrix().reshape(-1,1)
features = np.apply_along_axis(lambda x : x[0], 1, series)
test_dataset = TensorDataset(torch.from_numpy(features.astype('float32')), target)

test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=True, num_workers=0)

In [18]:
import torch.nn as nn
import torch.nn.functional as F

class MyMLP(nn.Module):
    def __init__(self):
        super(MyMLP, self).__init__()
        self.hidden1 = nn.Linear(4499,2056)
        self.hidden2 = nn.Linear(2056,1028)
        self.hidden3 = nn.Linear(1028,512)
        self.hidden4 = nn.Linear(512,256)
        self.hidden5 = nn.Linear(256,64)
        self.hidden6 = nn.Linear(64,16)
        self.out = nn.Linear(16,2)

    def forward(self, x):
        x = F.relu(self.hidden1(x))
        x = F.relu(self.hidden2(x))
        x = F.relu(self.hidden3(x))
        x = F.relu(self.hidden4(x))
        x = F.relu(self.hidden5(x))
        x = F.relu(self.hidden6(x))
        x = self.out(x)
        return x

In [19]:
import torch.nn as nn
import torch.nn.functional as F

class_weightage = torch.tensor([pos_count/neg_count,1-pos_count/neg_count])
print(class_weightage)
criterion = nn.CrossEntropyLoss(class_weightage)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tensor([0.0651, 0.9349])


In [20]:
#Load model
best_model = torch.load(dl_path + save_file)

In [21]:
#Test metrics
test_loss, test_accuracy, test_results = evaluate(best_model, device, test_loader, criterion)

Test: [0/323]	Time 0.046 (0.046)	Loss 0.5381 (0.5381)	Accuracy 87.000 (87.000)
Test: [10/323]	Time 0.028 (0.052)	Loss 0.5674 (0.6690)	Accuracy 93.000 (80.182)
Test: [20/323]	Time 0.067 (0.043)	Loss 0.6730 (0.6562)	Accuracy 81.000 (79.095)
Test: [30/323]	Time 0.035 (0.040)	Loss 0.5315 (0.6327)	Accuracy 87.000 (80.742)
Test: [40/323]	Time 0.014 (0.039)	Loss 0.4849 (0.6403)	Accuracy 90.000 (79.122)
Test: [50/323]	Time 0.041 (0.037)	Loss 0.6920 (0.6335)	Accuracy 78.000 (79.412)
Test: [60/323]	Time 0.017 (0.037)	Loss 0.6123 (0.6339)	Accuracy 84.000 (79.918)
Test: [70/323]	Time 0.032 (0.037)	Loss 0.5975 (0.6297)	Accuracy 90.000 (80.366)
Test: [80/323]	Time 0.031 (0.036)	Loss 0.6657 (0.6282)	Accuracy 90.000 (80.284)
Test: [90/323]	Time 0.015 (0.034)	Loss 0.5658 (0.6273)	Accuracy 81.000 (80.429)
Test: [100/323]	Time 0.023 (0.033)	Loss 0.6707 (0.6289)	Accuracy 75.000 (80.495)
Test: [110/323]	Time 0.029 (0.032)	Loss 0.6581 (0.6307)	Accuracy 90.000 (80.505)
Test: [120/323]	Time 0.018 (0.033)	Loss

In [22]:
Y_true=np.array(test_results)[:,0]
Y_pred=np.array(test_results)[:,1]
display_metrics("MLP",Y_pred,Y_true)

______________________________________________
Classifier: MLP
Accuracy: 0.7978290366350068
AUC: 0.6204180187144386
Precision: 0.13585858585858585
Recall: 0.41770186335403725
F1-score: 0.205030487804878
______________________________________________

